##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker를 사용한 객체 감지

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

이 colab 노트북에서는 [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) 라이브러리를 사용하여 모바일 기기의 이미지 내 샐러드를 감지할 수 있는 맞춤형 객체 감지 모델을 훈련하는 방법을 배웁니다.

Model Maker 라이브러리는 *전이 학습*을 사용하여 사용자 지정 데이터세트를 사용하여 TensorFlow Lite 모델을 훈련하는 프로세스를 단순화합니다. 고유한 사용자 지정 데이터세트로 TensorFlow Lite 모델을 재훈련하면 필요한 훈련 데이터의 양이 줄어들고 훈련 시간이 단축됩니다.

<a>Open Images Dataset V4</a>에서 생성된 공개적으로 사용 가능한 <em>샐러드</em> 데이터세트를 사용합니다.

데이터세트의 각 이미지에는 다음 클래스 중 하나로 레이블이 지정된 객체들이 포함되어 있습니다.

- 구운 음식
- 치즈
- 샐러드
- 해산물
- 토마토

데이터세트에는 객체의 레이블과 함께 각 객체의 위치를 지정하는 경계 상자가 포함되어 있습니다.

다음은 데이터세트의 예제 이미지입니다.

<br>


<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" class="">  


## 전제 조건


### 필요한 패키지 설치하기

우선 [GitHub 리포지토리](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)의 Model Maker 패키지와 평가에 사용할 pycocotools 라이브러리를 포함하여 필수 패키지를 설치합니다.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

필요한 패키지를 가져옵니다.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### 데이터세트 준비하기

여기에서는 AutoML [빠른 시작](https://cloud.google.com/vision/automl/object-detection/docs/edge-quickstart#preparing_a_dataset)과 동일한 데이터세트를 사용합니다.

*샐러드* 데이터세트는 `gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv`에서 사용할 수 있습니다.

여기에는 훈련용 이미지 175개, 검증용 이미지 25개, 테스트용 이미지 25개가 포함됩니다. 데이터세트에는 `Salad`, `Seafood`, `Tomato`, `Baked goods`, `Cheese` 등 5가지 클래스가 있습니다.

<br>

데이터세트는 CSV 형식으로 제공됩니다.

```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

- 각 행은 더 큰 이미지 내부에 국한된 객체에 해당하며 각 객체는 테스트, 훈련 또는 검증 데이터로 특정하게 지정됩니다. 이 노트북의 이후 단계에서 이것이 의미하는 바에 대해 자세히 알아볼 것입니다.
- 여기에 포함된 세 줄은 <code>gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg</code>에서 사용할 수 있는 <strong>동일한 이미지 안에 있는 세 개의 개별 객체</strong>를 나타냅니다.
- 각 행에는 `Salad`, `Seafood`, `Tomato` 등의 다른 레이블이 있습니다.
- 경계 상자는 왼쪽 상단과 오른쪽 하단 정점을 사용하여 각 이미지에 대해 지정됩니다.

다음은 이 세 줄을 시각화한 것입니다.

<br>


<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" width="400" hspace="100">

자체 CSV 파일을 준비하는 방법과 유효한 데이터세트를 만들기 위한 최소 요구 사항에 대해 자세히 알아보려면 [훈련 데이터 준비](https://cloud.google.com/vision/automl/object-detection/docs/prepare) 가이드에서 자세한 내용을 참조하세요.

Google Cloud를 처음 사용하는 경우 `gs://` URL이 무엇을 의미하는지 궁금할 수 있습니다. 이것은 [Google Cloud Storage](https://cloud.google.com/storage)(GCS)에 저장된 파일의 URL입니다. GCS의 파일을 공개하거나 [클라이언트를 인증](https://cloud.google.com/storage/docs/authentication#libauth)하면 Model Maker가 로컬 파일과 유사하게 해당 파일을 읽을 수 있습니다.

그러나 Model Maker를 사용하기 위해 Google Cloud에 이미지를 보관할 필요는 없습니다. CSV 파일에서 로컬 경로를 사용할 수 있으며 Model Maker가 제대로 작동합니다.

## 빠른 시작

객체 감지 모델을 학습시키는 6단계는 다음과 같습니다.

**1단계. 객체 감지 모델 아키텍처를 선택합니다.**

이 튜토리얼은 EfficientDet-Lite0 모델을 사용합니다. EfficientDet-Lite[0-4]는 [EfficientDet](https://arxiv.org/abs/1911.09070) 아키텍처에서 파생된 모바일/IoT 친화적인 객체 감지 모델군입니다.

다음은 각 EfficientDet-Lite 모델의 성능을 서로 비교한 것입니다.

모델 아키텍처 | 크기(MB)* | 지연 시간(밀리초)** | 평균 정밀도***
--- | --- | --- | ---
EfficientDet-Lite0 | 4.4 | 37 | 25.69%
EfficientDet-Lite1 | 5.8 | 49 | 30.55%
EfficientDet-Lite2 | 7.2 | 69 | 33.97%
EfficientDet-Lite3 | 11.4 | 116 | 37.70%
EfficientDet-Lite4 | 19.9 | 260 | 41.96%

<i>* 정수 양자화 모델의 크기.<br> ** 지연 시간은 CPU에서 4개의 스레드를 사용하여 Pixel 4에서 측정했습니다.<br> *** 평균 정밀도는 COCO 2017 검증 데이터세트의 mAP(mean Average Precision)입니다.</i>


In [ ]:
spec = model_spec.get('efficientdet_lite0')

**2단계. 데이터세트를 로드합니다.**

Model Maker는 CSV 형식의 입력 데이터를 가져옵니다. `object_detector.DataLoader.from_csv` 메서드를 사용하여 데이터세트를 로드하고 훈련, 검증 및 테스트 이미지로 분할합니다.

- 훈련 이미지: 이러한 이미지는 샐러드 재료를 인식하도록 객체 감지 모델을 훈련하는 데 사용됩니다.
- 검증 이미지: 학습 과정에서 모델이 인식하지 못한 이미지입니다. [과적합](https://en.wikipedia.org/wiki/Overfitting)을 피하기 위해 훈련을 중단해야 하는 시점을 결정하는 데 사용합니다.
- 테스트 이미지: 이러한 이미지는 최종 모델 성능을 평가하는 데 사용됩니다.

Google Cloud Storage에서 직접 CSV 파일을 로드할 수 있지만 Model Maker를 사용하기 위해 Google Cloud에 이미지를 보관할 필요는 없습니다. 컴퓨터에서 로컬 CSV 파일을 지정할 수 있으며 Model Maker이 제대로 작동합니다.

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**3단계. 훈련 데이터로 TensorFlow 모델을 훈련합니다.**

- EfficientDet-Lite0 모델은 기본적으로 `epochs = 50`을 사용합니다. 즉, 훈련 데이터세트를 50번 거치게 됩니다. 훈련 중에 유효성 검사 정확도를 확인하고 과적합을 방지하기 위해 일찍 중지할 수 있습니다.
- 여기서 `batch_size = 8`로 설정하면 훈련 데이터세트의 175개 이미지를 거치는 데 21단계가 필요함을 알 수 있습니다.
- `train_whole_model=True`를 설정하여 정확도를 향상시키기 위해 헤드 레이어를 훈련하는 대신 전체 모델을 미세 조정합니다. 단점은 모델을 훈련하는 데 시간이 더 오래 걸릴 수 있다는 것입니다.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

**4단계. 테스트 데이터로 모델을 평가합니다.**

훈련 데이터세트의 이미지를 사용하여 객체 감지 모델을 훈련한 후, 테스트 데이터세트의 나머지 25개 이미지를 사용하여 이전에 본 적이 없는 새 데이터에 대해 모델이 어떻게 작동하는지 평가합니다.

기본 배치 크기는 64이므로 테스트 데이터세트의 25개 이미지를 거치는 데 1단계가 걸립니다.

평가 메트릭은 [COCO](https://cocodataset.org/#detection-eval)와 동일합니다.

In [ ]:
model.evaluate(test_data)

**5단계. TensorFlow Lite 모델로 내보냅니다.**

양자화된 모델을 내보낼 폴더를 지정하여 훈련된 객체 감지 모델을 TensorFlow Lite 형식으로 내보냅니다. 기본 훈련 후 양자화 기술은 전체 정수 양자화입니다.

In [ ]:
model.export(export_dir='.')

**6단계. TensorFlow Lite 모델을 평가합니다.**

TFLite로 내보낼 때 몇 가지 요인이 모델 정확도에 영향을 줄 수 있습니다.

- [양자화](https://www.tensorflow.org/lite/performance/model_optimization)는 정확도가 약간 떨어지는 대신 모델 크기를 4배 줄이는 데 도움을 줍니다.
- 원래 TensorFlow 모델은 후처리를 위해 클래스별 [NMS(non-max supression)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH)를 사용하는 반면, TFLite 모델은 훨씬 빠르지만 정확도는 떨어지는 전역 NMS를 사용합니다. Keras는 최대 100개의 감지를 출력하고 tflite는 최대 25개의 감지를 출력합니다.

따라서 내보낸 TFLite 모델을 평가하고 정확도를 원래 TensorFlow 모델과 비교해야 합니다.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

Colab의 왼쪽 사이드바를 사용하여 TensorFlow Lite 모델 파일을 다운로드할 수 있습니다. `model.tflite` 파일을 마우스 오른쪽 버튼으로 클릭하고 `Download`를 선택하여 로컬 컴퓨터에 다운로드합니다.

이 모델은 [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector)의 [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview)를 사용하여 Android 또는 iOS 앱에 통합할 수 있습니다.

모델이 작동 앱에서 어떻게 사용되는지 자세히 알아보려면 [TFLite 객체 감지 샘플 앱](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android)을 참조하세요.

*참고: Android Studio 모델 바인딩은 아직 객체 감지를 지원하지 않으므로 TensorFlow Lite 작업 라이브러리를 사용하세요.*

## (선택 사항) 이미지에서 TFLite 모델 테스트

인터넷의 이미지를 사용하여 훈련된 TFLite 모델을 테스트할 수 있습니다.

- 아래 `INPUT_IMAGE_URL`을 원하는 입력 이미지로 바꿉니다.
- `DETECTION_THRESHOLD`를 조정하여 모델의 감도를 변경합니다. 임계값이 낮을수록 모델이 더 많은 객체를 픽업하지만 잘못된 감지도 더 많이 발생시킵니다. 한편, 임계값이 높을수록 모델은 확신을 가지고 감지한 객체만 픽업합니다.

현재, Python에서 모델을 실행하려면 일부 상용구 코드가 필요하지만 모델을 모바일 앱에 통합하려면 몇 줄의 코드만 있으면 됩니다.

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
#@title Run object detection and show the detection results

INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

## (선택 사항) Edge TPU용으로 컴파일

이제 양자화된 EfficientDet Lite 모델이 있으므로 컴파일하고 [Coral EdgeTPU](https://coral.ai/)로 배포할 수 있습니다.

**1단계. EdgeTPU 컴파일러 설치하기**

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

**2단계. Edge TPU 수를 선택하고 컴파일**

EdgeTPU에는 모델 매개변수를 캐싱하기 위한 8MB의 SRAM이 있습니다([추가 정보](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching)). 즉, 8MB보다 큰 모델의 경우 모델 매개변수를 통해 전송할 수 있도록 추론 시간이 늘어납니다. 이렇게 되지 않도록 하는 한 가지 방법은 [모델 파이프라이닝](https://coral.ai/docs/edgetpu/pipeline/)으로, 모델을 전용 EdgeTPU를 가질 수 있는 세그먼트로 분할하는 것입니다. 이렇게 하면 지연 시간이 크게 개선될 수 있습니다.

아래 표는 사용할 Edge TPU 수에 대한 참조로 사용할 수 있습니다. 중간 텐서는 온칩 메모리에 맞지 않기 때문에 더 큰 모델은 단일 TPU에 대해 컴파일되지 않습니다.

모델 아키텍처 | 최소 TPU | 권장 TPU
--- | --- | ---
EfficientDet-Lite0 | 1 | 1
EfficientDet-Lite1 | 1 | 1
EfficientDet-Lite2 | 1 | 2
EfficientDet-Lite3 | 2 | 2
EfficientDet-Lite4 | 2 | 3

In [ ]:
NUMBER_OF_TPUS =  1#@param {type:"number"}

!edgetpu_compiler model.tflite --num_segments=$NUMBER_OF_TPUS

**3단계. 모델 다운로드, 실행**

모델이 컴파일되면 이제 객체 감지를 위해 EdgeTPU에서 실행할 수 있습니다. 먼저 Colab의 왼쪽 사이드바를 사용하여 컴파일된 TensorFlow Lite 모델 파일을 다운로드합니다. `model_edgetpu.tflite` 파일을 마우스 오른쪽 버튼으로 클릭하고 `Download` 를 선택하여 로컬 컴퓨터에 다운로드합니다.

이제 원하는 방식으로 모델을 실행할 수 있습니다. 감지의 예는 다음과 같습니다.

- [pycoral 감지](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py)
- [기본 TFLite 감지](https://github.com/google-coral/tflite/tree/master/python/examples/detection)
- [비디오 감지 예제](https://github.com/google-coral/examples-camera)
- [libcoral C++ API](https://github.com/google-coral/libcoral)


## 고급 사용법

이 섹션에서는 모델 조정 및 훈련 하이퍼 매개변수와 같은 고급 사용 주제를 다룹니다.

### 데이터세트 로드

#### 자신의 데이터 로드

이 튜토리얼을 진행하기 위해 자신의 데이터세트를 업로드할 수 있습니다. Colab의 왼쪽 사이드바를 사용하여 데이터세트를 업로드하세요.


<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" width="1000" hspace="0" alt="파일 업로드">

데이터세트를 클라우드에 업로드하지 않으려면 [가이드](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)에 따라 로컬에서 라이브러리를 실행할 수도 있습니다.

#### 다른 데이터 형식으로 데이터 로드

Model Maker 라이브러리는 <a>PASCAL VOC</a> 형식으로 데이터를 로드하는 <code>object_detector.DataLoader.from_pascal_voc</code> 메서드도 지원합니다. [makesense.ai](https://www.makesense.ai/) 및 [LabelImg](https://github.com/tzutalin/labelImg)는 이미지에 주석을 달 수 있고 주석을 PASCAL VOC 데이터 형식의 XML 파일로 저장할 수 있는 도구입니다.

```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### EfficientDet 모델 하이퍼 매개변수 사용자 지정

조정할 수 있는 모델 및 훈련 파이프라인 매개변수는 다음과 같습니다.

- `model_dir`: 모델 체크포인트 파일을 저장할 위치입니다. 설정하지 않으면 임시 디렉터리가 사용됩니다.
- `steps_per_execution`: 훈련 실행당 단계 수입니다.
- `moving_average_decay`: 부동이며, 훈련된 매개변수의 이동 평균을 유지하는 데 사용할 감쇠입니다.
- `var_freeze_expr`: 훈련 중에 동일하게 유지됨을 의미하는 고정될 변수의 접두사 이름을 매핑하는 정규식입니다. 보다 구체적으로, 코드베이스에서 `re.match(var_freeze_expr, variable_name)`를 사용하여 고정할 변수를 매핑합니다.
- `tflite_max_detections`: 정수이며 기본적으로 25입니다. TFLite 모델의 최대 출력 감지 수입니다.
- `strategy`: 사용할 배포 전략을 지정하는 문자열입니다. 허용되는 값은 'tpu', 'gpus', None입니다. 'tpu'는 TPUSStrategy를 사용하는 것을 의미합니다. 'gpus'는 다중 GPU에 MirroredStrategy를 사용하는 것을 의미합니다. None인 경우 OneDeviceStrategy와 함께 TF 기본값을 사용합니다.
- `tpu`: 학습에 사용할 Cloud TPU입니다. Cloud TPU를 만들 때 사용된 이름이거나 grpc://ip.address.of.tpu:8470 URL이어야 합니다.
- `use_xla`: 전략이 tpu가 아니더라도 XLA를 사용합니다. 전략이 tpu이면 항상 XLA를 사용하고 이 플래그는 효과가 없습니다.
- `profile`: 프로필 모드를 활성화합니다.
- `debug`: 디버그 모드를 활성화합니다.

조정할 수 있는 다른 매개변수는 [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170)에 표시됩니다.

예를 들어, `var_freeze_expr='efficientnet'`을 설정할 수 있습니다. 이는 이름 접두사 `efficientnet`이 있는 변수를 고정합니다(기본값은 `'(efficientnet|fpn_cells|resample_p6)'`). 이를 통해 모델은 훈련할 수 없는 변수를 동결하고 훈련하는 동안 그 값을 동일하게 유지할 수 있습니다.

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### 모델 아키텍처 변경

`model_spec`을 변경하여 모델 아키텍처를 변경할 수 있습니다. 예를 들어 `model_spec`을 EfficientDet-Lite4 모델로 변경합니다.

```python
spec = model_spec.get('efficientdet_lite4')
```

### 훈련 하이퍼 매개변수 조정

`create` 함수는 Model Maker 라이브러리가 모델을 생성하는 데 사용하는 드라이버 함수입니다. `model_spec` 매개변수는 모델 사양을 정의합니다. `object_detector.EfficientDetSpec` 클래스가 현재 지원됩니다. `create` 기능은 다음 단계로 구성됩니다.

1. `model_spec`에 따라 객체 감지를 위한 모델을 생성합니다.
2. 모델을 학습시킵니다. 기본 epochs 및 기본 배치 크기는 `model_spec` 객체의 `epochs` 및 `batch_size` 변수에 의해 설정됩니다. 모델 정확도에 영향을 미치는 `epochs` 및 `batch_size`와 같은 훈련 하이퍼 매개변수를 조정할 수도 있습니다. 예를 들면 다음과 같습니다.

- `epochs`: 정수이며, 기본적으로 50입니다. Epoch가 더 많으면 정확도가 개선되지만 과적합으로 이어질 수 있습니다.
- `batch_size`: 정수이며 기본적으로 64입니다. 한 훈련 단계에서 사용할 샘플 수입니다.
- `train_whole_model`: 부울이며 기본적으로 False입니다. true이면 전체 모델을 훈련시킵니다. 그렇지 않으면 `var_freeze_expr`과 일치하지 않는 레이어만 훈련합니다.

예를 들어, 더 적은 epoch와 헤드 레이어만 사용하여 훈련할 수 있습니다. 더 나은 결과를 위해 epoch 수를 늘릴 수 있습니다.

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### 다른 형식으로 내보내기

내보내기 형식은 다음 중 하나 또는 목록일 수 있습니다.

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`


기본적으로, 나중에 온디바이스 ML 애플리케이션에서 사용할 수 있도록 모델 [메타데이터](https://www.tensorflow.org/lite/models/convert/metadata)가 포함된 TensorFlow Lite 모델 파일만 내보냅니다. 레이블 파일은 메타데이터에 포함됩니다.

많은 온디바이스 ML 애플리케이션에서 모델 크기는 중요한 요소입니다. 따라서 모델을 더 작게 만들고 더 빠르게 실행할 수 있도록 모델을 양자화하는 것이 좋습니다. EfficientDet-Lite 모델의 경우, 기본적으로 모델을 양자화하기 위해 전체 정수 양자화가 사용됩니다. 자세한 내용은 [훈련 후 양자화](https://www.tensorflow.org/lite/performance/post_training_quantization)를 참조하세요.

```python
model.export(export_dir='.')
```

더 나은 검사를 위해 모델과 관련된 다른 파일을 내보내도록 선택할 수도 있습니다. 예를 들어 다음과 같이 저장된 모델과 레이블 파일을 모두 내보냅니다.

```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### TensorFlow Lite 모델에서 훈련 후 양자화 사용자 정의하기

[훈련 후 양자화](https://www.tensorflow.org/lite/performance/post_training_quantization)는 모델 정확도를 약간만 떨어트리면서 모델 크기와 추론 지연 시간을 줄이는 동시에 CPU 및 하드웨어 가속기의 추론 속도도 개선할 수 있는 변환 기술입니다. 따라서 모델을 최적화하는 데 널리 사용됩니다.

Model Maker 라이브러리는 모델을 내보낼 때 기본 훈련 후 양자화 기술을 적용합니다. 훈련 후 양자화를 사용자 지정하려는 경우 Model Maker는 [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig)를 사용하여 여러 훈련 후 양자화 옵션도 지원합니다. float16 양자화를 예로 들어보겠습니다. 먼저 양자화 구성을 정의합니다.

```python
config = QuantizationConfig.for_float16()
```

그런 다음 이러한 구성을 가진 TensorFlow Lite 모델을 내보냅니다.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# 더 읽어보기

기술적인 세부 사항을 알아보려면 [객체 감지](https://www.tensorflow.org/lite/examples/object_detection/overview) 예제를 읽어볼 수 있습니다. 자세한 내용은 다음을 참조하세요.

- TensorFlow Lite Model Maker [가이드](https://www.tensorflow.org/lite/models/modify/model_maker) 및 [API 참조](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)
- 작업 라이브러리: 배포용 [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector)
- 엔드 투 엔드 참조 앱: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios) 및 [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi)
